## News

Мы решили торговать на бирже, причем так, чтобы решение о покупке акций принимала нейросеть на основе последних новостей о той или иной компании. Для этого нужно научиться классифицировать все новости. Дана база новостей из разных источников news.csv. Необходимо написать классифицирующую модель новостей по источникам.

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')


data = pd.read_csv('news(1).csv', engine='python', error_bad_lines=False)
data.drop('Unnamed: 0', axis=1, inplace=True)
df_valid = data[data.source.isna()]
df = data[~data.source.isna()]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Андрей\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Андрей\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from nltk.corpus import stopwords
import re
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer

# Проведем предобработку и векторизацию текста.

stop_words = stopwords.words('russian')
corpus = df['news'].tolist()
corpus = [" ".join(re.findall(r"(?u)\b\w\w+\b", i)) for i in corpus]

stemmer = SnowballStemmer(language='russian')
corpus = [stemmer.stem(w) for w in corpus]

vectorizer = CountVectorizer(stop_words = stop_words)
X_train = vectorizer.fit_transform(corpus)
feature_names = vectorizer.get_feature_names()

In [8]:
y_train

0        1.0
1        1.0
2        0.0
3        2.0
4        2.0
        ... 
18355    1.0
18356    1.0
18357    1.0
18358    0.0
18359    1.0
Name: source, Length: 18360, dtype: float64

In [10]:
# Построим модель логистической регрессии и рассчитаем score при помощи кросс-валидации.

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

y_train = df['source']
model = LogisticRegression(solver='sag')
model.fit(X_train, y_train)

cross_val_score(model, X_train, y_train, cv=5)

array([0.90087146, 0.90686275, 0.89460784, 0.89678649, 0.89297386])

In [11]:
# Проведем преобразование тестового датасета, сделаем предсказание и сохраним результаты.

corpus_valid = df_valid['news'].tolist()
corpus_valid = [" ".join(re.findall(r"(?u)\b\w\w+\b", i)) for i in corpus]

corpus_valid = [stemmer.stem(w) for w in corpus_valid]

vectorizer = CountVectorizer(stop_words = stop_words)
X_test = vectorizer.fit_transform(corpus)

y_pred = model.predict(X_test)
precision = pd.DataFrame(y_pred)
precision.head()
precision.to_csv('submission', header=False, index=False)